# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

This is classification. The difference is that classification tries to assign a given data point to one of multiple classes, where the classes are pre-defined, while regression tries to predict some (usually continuous) response based on a given data point. In this case, we have two classes - "likely to fail" and "not likely to fail" - and we need to determine which of these two classes each student in the testing data falls into. By definition, that is classification.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [2]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [7]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.shape[1] - 1 # Subtract 1 to remove the count for the label column.

# TODO: Calculate passing students
n_passed = (student_data["passed"]).value_counts()[0]

# TODO: Calculate failing students
n_failed = (student_data["passed"]).value_counts()[1]

# TODO: Calculate graduation rate
grad_rate = float(n_passed)/(float(n_passed) + float(n_failed))*100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [5]:
# TODO: Import any additional functionality you may need here
import random
random.seed(1993)

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
choose = np.random.choice(395, num_train, replace=False) # This gives the row selections for the random shuffle.

X_train = X_all.iloc[choose]
X_test = X_all.drop(X_all.index[[choose]], axis=0)
y_train = y_all.iloc[choose]
y_test = y_all.drop(y_all.index[[choose]], axis=0)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. What are the general applications of each model? What are their strengths and weaknesses? Given what you know about the data, why did you choose these models to be applied?*

The three supervised learning models that I am going to use for this problem are Naive Bayes, decision trees, and support vector machines. I describe each of them and why they are plausible candidates for this dataset below.

Naive Bayes is a classification algorithm that is popular in machine learning because it frequently performs as well as or better than more sophisticated techniques. It is commonly used in text classification (determining out of a collection of texts which texts belong to which group) and medical diagnosis. It is especially useful in performing classification when we have reason to believe that the value of one variable in the dataset tells us little about the value of another variable in the same dataset, and/or when the dataset is high dimensional. Naive Bayes models are simple in the assumptions they make: they assume that each of the attribute variables are probabilistically independent of each other. For example, in the context of this dataset, the probability of being female given that a student graduates and the probability of having Internet access given that a student graduates are independent; and the same relationship holds between all of the variables. Let me make what I just said as clear as possible: the phrase "...the same relationship holds between all of the variables" is not a statement about what is actually the case with this dataset. It is a statement about what Naive Bayes ASSUMES to be the case. Again: I am not saying that the relationship of independence DOES hold between all variables, but that the Naive Bayes model assumes this relationship. Even though this may not represent the actual variable relationships, this model seems useful for the data for reasons I'll explain below. Naive Bayes also has the advantage of being computationally faster than most other models, because 1) it has to estimate comparatively few terms (the number of features multiplied by the number of classes), and 2) the probabilities it estimates are one-dimensional, rather than being higher-dimensional and thus requiring more data and more computing time. The weakness of the model comes from the same source as its strength: its strong conditional independence assumption may provide a bad fit to the data if some of the variables actually are dependent on one another. This may lead to model bias - a lack of good fit stemming from the model's inability to capture the contours of the data when certain variables are not probabilistically independent of one another. I chose to use Naive Bayes with this dataset because this dataset is high-dimensional - it has 395 observations and 30 features. Naive Bayes estimates only one-dimensional probabilities, and thus avoids the curse of dimensionality. (That is, not as much data is needed to estimate these one-dimensional probabilities as would be needed for longer joint probabilities.) This is also why Naive Bayes can be trained on relatively small datasets.

Decision trees are often used in operations research to identify a plan that's likely to produce a successful result. They sort the data based on a hierarchical model that is shaped like a tree, starting with a single node and branching out into multiple other nodes, which then do the same, until the tree is complete. The observations in the dataset are contained in the nodes at the ends of the branches at the bottom of the tree. In general, decision trees are useful for implicitly performing variable screening, and for building models that fit data with nonlinear relationships between variables; they're well-liked because they run very quickly and are easy to interpret. Decision trees are built from splitting the data into categories at each node based on their value for a single variable. At each level of the tree, these splits are made to maximize "information gain" - a measure of how well the data are sorted after the split compared to before. Decision trees have the advantage of being relatively easy to understand and visualize, but are prone to overfitting the training data. A weakness of decision trees is that they are quite sensitive to noise in the data and are very prone to overfitting the training set; this usually has to be compensated for by either restricting how specific a tree can become in its lower branches, or by pruning a completed tree. Care must be used to avoid making too many splits in the tree, as doing so may lead to a tree that fits the observed data well but generalizes to new data poorly. I chose decision trees for this dataset because splitting by information gain seems like an intiuitive way to approach a classification problem where we have 1) a large number of variables, and 2) little up-front idea of which ones might be most important. A graph of the tree would provide a good visualization of what factors make it more likely that a student will fail to graduate, and would show them in order of importance.

Support vector machines are often used with high-dimensional data (making them popular in bioinformatics or image detection), and/or with data where nonlinear relationships exist between the variables. They are often used in life sciences to identify proteins, or in text mining to identify hand-written characters. SVMs estimate a decision boundary between points of one class and points of another, based on higher-dimensional separating hyperplanes between the points. These boundaries are created primarily based on the variable differences between points of different classes that are closest to one another. The name of the model comes from the way that the hyperplane is constructed - support vectors are the points closest to the supporting hyperplane, and they determine its shape. The movement of one of these points even by a small amount can change the decision boundary significantly. Conversely, points which are farther away from the space(s) between different classes are given much less weight (often none, for practical purposes) in determining the shape of the hyperplane. Support vector machines make predictions based on what side of the decision boundary testing points fall, with one side representing one of the classes and the other side representing the other class. If the points of the two classes are linearly separable, the hyperplane will take the form of a line when projected into two-dimensional space. If the classes are not linearly separable (if there is no line that can be drawn between points of both classes in two-dimensional space that perfectly separates them), a non-linear kernel can be used for the hyperplane, that will take the form of some sort of curve when projected into two-dimensional space. The advantage of support vector machines for this dataset is that they are designed specifically for binary classification, which is what we're trying to do with this data. Additionally, they're useful in teasing out the small differences between students who graduate and those who don't, because of the weight they give in constructing the decision boundary to those small differences between students in the two classes who are otherwise similar. A disadvantage comes from the curse of dimensionality: the separating hyperplane may be very high-dimensional, and the number of data points needed to estimate the hyperplane grows exponentially with the number of dimensions.

References I consulted for this write-up include:

"4 key advantages of using decision trees," http://www.simafore.com/blog/bid/62333/4-key-advantages-of-using-decision-trees-for-predictive-analytics
Columbia Lecture on Support Vector Machines, http://www.cs.columbia.edu/~kathy/cs4701/documents/jason_svm_tutorial.pdf

Sklearn documentation on Naive Bayes, http://scikit-learn.org/stable/modules/naive_bayes.html

Naive Bayes Wiki article, https://en.wikipedia.org/wiki/Naive_Bayes_classifier

Decision tree Wiki article, https://en.wikipedia.org/wiki/Decision_tree

Support vector machine Wiki article, https://en.wikipedia.org/wiki/Support_vector_machine




### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_classifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [7]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [12]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
# from sklearn import model_B
# from sklearn import model_C

from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm

import random
random.seed(1993)
# CITATION FOR ABOVE CODE BLOCK: Using a random seed to make sure the same values were generated by np.random.choice()
# each time was my idea based on my previous experience in R, but I found the way of writing this in Python
# on StackExchange, in the comment by Jon Clements on this thread. http://stackoverflow.com/questions/11526975/set-random-seed-programwide-in-python


# TODO: Initialize the three models
clf_A = GaussianNB()
clf_B = DecisionTreeClassifier(criterion="entropy")
clf_C = svm.SVC()

# TODO: Set up the training set sizes
X_train_100 = X_train.iloc[:100]
y_train_100 = y_train.iloc[:100]

X_train_200 = X_train.iloc[:200]
y_train_200 = y_train.iloc[:200]

X_train_300 = X_train
y_train_300 = y_train

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)

# To avoid having to type out the same command 9 times, I put the names of the different train sizes for X and y
# and the different classifiers into the lists below so that I could loop over them.

clf_list = [clf_A, clf_B, clf_C]
train_size_list = [(X_train_100, y_train_100), (X_train_200, y_train_200), (X_train_300, y_train_300)]

for i in clf_list:
    for j in train_size_list:
        print "\n"
        train_predict(i, j[0], j[1], X_test, y_test)
        




Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0016 seconds
Made predictions in 0.0012 seconds.
F1 score for training set: 0.7667.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.6261.


Training a GaussianNB using a training set size of 200. . .
Trained model in 0.0010 seconds
Made predictions in 0.0004 seconds.
F1 score for training set: 0.7491.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.7826.


Training a GaussianNB using a training set size of 300. . .
Trained model in 0.0008 seconds
Made predictions in 0.0004 seconds.
F1 score for training set: 0.7961.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.7626.


Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0009 seconds
Made predictions in 0.0002 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.6435.


Training a DecisionTreeClassifier using a training set siz

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Gaussian Naive Bayes**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |           0.0012              |0.0003               |0.7667            |0.6261         | 
| 200               |        0.0004          |     0.0003                |0.7491            |0.7826         |
| 300               |          0.0004               | 0.0003             |0.7961            |0.7626         |

** Classifer 2 - Decision Tree Classifier**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |  0.0002                     |0.0002                  |1                  |0.6435              |
| 200               |  0.0002                     |0.0002                  |1                  |0.5124              |
| 300               |  0.0002                    |0.0002                 |1                  |0.7704              |

** Classifer 3 - Support Vector Machine**  

| Training Set Size | Prediction Time (train) | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |  0.0008                   |0.0007                  |0.8514               |0.7600         |
| 200               |  0.0023                   |0.00012                  |0.8121            |0.8125         |
| 300               |  0.0052                    |0.0026                  |0.8528            |0.8212            |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Chosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

To determine which model we should use, we need to know which one consistently gives the best F1 test score. The F1 score is the weighted average of the model's precision (the probability, given the model's classification, that a data point is actually in the predicted class) and recall (the probability, given that the data is in a given class, that the model classifies it correctly). It works as a measure of how well the model classifies data, and ranges from 0 to 1, with 0 being worst and 1 being best. We look at the F1 test score rather than the F1 train score because the latter is prone to overfitting; it may be artificially inflated because the model has already seen the data by which it's making a prediction. The F1 test score is a better approximation to how the model performs with unseen data.

There are three considerations that need to be made. First, which model has the best F1 test score overall when trained on the full training data (300)? Second, which model gives the most consistent F1 score for the different training sizes? The second consideration is important because the actual population of students may be much larger than the given 395, and we need to know whether our model is likely to fluctuate widely in its predictions based on the size of the training set, especially since getting data on the majority of a large school may be unfeasible. Sampling students may be costly, and so we need to know that the model will be relatively consistent in the quality of its predictions even if our training set is smaller than ideal. Third, which model has the best computational run-time? Depending on the size of the sample and the number of variables, run-time might make a difference in how easy the analysis to perform. By the first two metrics, the support vector machine (SVM) model is clearly the best. It has the highest overall F1 score on the full training data, and is much more consistent in its score across the training set sizes than the decision tree or naive Bayes models. The lower F1 scores of those two models indicate that they are not as good a fit to the data, and the variability of their F1 scores by data size indicates that they are more sensitive to the size of the sample than the support vector model.

When taking time into account, the SVM model is slower (on average, by a factor of 10) than the decision tree or Naive Bayes models on training prediction time, and slightly slower (but about 0.0001 seconds on average) than these models in the testing prediction time. However, this does not provide reason for rejecting the SVM model in favor of one of the other two. All of the models differ in runtime by less than the tenth part of a second. If the school expected to be using a very large (say, 10,000+ students) dataset, and/or had very slow computers available, it might be preferable to use the decision tree classifier. But neither of these seem likely; given limited resources, it's very improbable that the sample size will be large enough to actually slow down an average computer by more than a few seconds. Runtime is a more important issue when dealing with large datasets, but 395 observations is not nearly large enough to justify prioritizing runtime over F1 score, especially when the runtime difference is miniscule. I conclude that the SVM model seems to be the best for this data.



### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. For example if you've chosen to use a decision tree or a support vector machine, how does the model go about making a prediction?*

Support vector machines rely on the concept of a decision boundary. Decision boundaries are criteria by which we classify data points. If the available points are graphed in two-dimensional space, the decision boundary is the line or curve that best separates the points by their classes. The decision boundary is constructed primarily based on the points of different classes which are closest to one another: for instance, if a student who is known to have not graduated falls close on the graph to a student who did graduate, the differences in variable values between these two students will be given much more weight in constructing the decision boundary than the differences between failing and graduating students who differed widely in most respects. The boundary gives the greatest weight to the small differences between data points that are otherwise similar.

Once the decision boundary is created, it is used to classify the available data points. The points on the two-dimensional graph which lie on one side of the boundary are put in one class, and the points on the other side of the boundary are put into the other class. While this can still lead to some misclassification errors, SVMs are constructed to minimize these so that the boundary is as accurate as possible in separating the two classes.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [15]:
# TODO: Import 'gridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

# NOTE HERE: Is this a typo below? It says "Create the parameters list," but the instructions above say to create
# a dictionary.

# TODO: Create the parameters list you wish to tune
parameters = {"C" : [0.001, 0.01, 0.1, 1.0, 5.0, 10.0, 100.0], "kernel" : ["rbf", "sigmoid"]}

# TODO: Initialize the classifier
clf = svm.SVC()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label="yes")

######################## CITATION FOR ABOVE CODE BLOCK ########################
# Originally, I had written the code block above as:
#
# f1_scorer = make_scorer(f1_score)
#
# Doing this led to a value error in fitting the grid search model. I used Google to search for the meaning of the
# error message, and came across a description of this same problem on StackExchange. The top-voted comment, by
# a user named Olologin, suggested using the argument "pos_label='yes'" in the make_scorer function because the
# classes are coded as "yes" and "no" rather than 1 and 0. I used this suggestion in altering my code.
# The link to this thread is here: http://stackoverflow.com/questions/34221712/grid-search-with-f1-as-scoring-function-several-pages-of-error-message

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0051 seconds.
Tuned model has a training F1 score of 0.8528.
Made predictions in 0.0019 seconds.
Tuned model has a testing F1 score of 0.8212.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

The final F1 score for training is 0.8528, and the F1 score for testing is 0.8212. These are the same values that were achieved by the untuned model, which indicates that the SVC's default values for C and kernel (1.0 and "rbf", respectively) were already best suited to this problem. Even though the F1 scores for the tuned model are the same, this does not mean that tuning was superfluous - the default values may be best (there's a reason why they're set as the defaults), but until we've tuned the parameters, we don't know for sure if they are. Tuning the model like I did above gives us better confidence in the fit of the model, even if the F1 scores are the same.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.